In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random 
import re
from scipy import spatial
%matplotlib inline
import math 
from collections import Counter 
import nltk
from operator import itemgetter 
nltk.download('wordnet')
from scipy.spatial import distance
import sys
from tqdm import tqdm
import itertools
from itertools import permutations 
import pylab
import tqdm 
import time
from textblob import TextBlob
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
#graduate courses list
Univ1_dataset=pd.read_csv('Univ-1.csv')

In [ ]:
Univ1_dataset.head()

In [ ]:
Univ1_dataset.dtypes

In [ ]:
Univ1_dataset=Univ1_dataset.applymap(str)

In [ ]:
Univ1_dataset.head()

In [ ]:
#Checking the datatypes of the dataset
Univ1_dataset.dtypes

In [ ]:
#removing the non-ascii characters present in the course and prerequisite column on university dataset
Univ1_dataset['Course']=Univ1_dataset['Course'].str.replace('\xa0',' ')
Univ1_dataset['Prerequisite']=Univ1_dataset['Prerequisite'].str.replace('\xa0',' ')

#removing the quotes and backslash from the course column of the dataset
def quotes(text):
  text = text.strip('\"')
  text=text.strip()
  return text
Univ1_dataset['Course']=Univ1_dataset['Course'].apply(quotes)


In [ ]:
#tokenizing the prerquisites

Univ1_dataset['Prerequisite'] = Univ1_dataset['Prerequisite'].str.split(',')

In [ ]:
#removing extra spaces in the pre list
Univ1_dataset['Prerequisite'] = Univ1_dataset['Prerequisite'].apply(lambda x:[w.lstrip() for w in x])

In [ ]:
Univ1_dataset[Univ1_dataset['Course']=='CS 643'].values

In [ ]:
Univ1_dataset.head()

In [ ]:
#creating courses list and prerequsites list from the Univ1_dataset
course=list(Univ1_dataset['Course'])
pre=list(Univ1_dataset['Prerequisite'])

In [ ]:
#removing the prereq which are not in Univerity 1 graduate courses

for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j not in course:
      list_1[index]='nan'

In [ ]:
#removing the 'nan' values
for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j=='nan':
      list_1.remove('nan')
for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j=='nan':
      list_1.remove('nan')
for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j=='nan':
      list_1.remove('nan')

In [ ]:
#data dict represents key as course from Univ-1 graduate courses and its prerequisite as value
data_dict={}
for i in range(len(course)):
  data_dict[course[i]]=pre[i]

In [ ]:
#removing redundancies in the data dictionary
data_dict['BNFO 700B']=[]
data_dict['BNFO 701B']=[]
data_dict['IS 701B']=[]
data_dict['CS 735']=['CS 675', 'CS 634', 'CS 643', 'CS 644']
data_dict['CS 726']=[]
data_dict['CS 700B']=[]
data_dict['CS 635']=['CS 505','CS 510']
data_dict['CS 640']=['CS 540']
data_dict['CS 646']=['CS 645','CS 646','CS 696']
data_dict['CS 650']=['CS 510']
data_dict['CS 668']=['CS 610','CS 650']
data_dict['CS 696']=['CS 652','CS 656']
data_dict['CS 701C']=[]
data_dict['CS 725']=[]
data_dict['CS 790A']=['CS 791']
data_dict['CS 790B']=['CS 791']
data_dict['CS 790C']=['CS 791']
data_dict['IS 591']=[]
data_dict['IS 592']=[]
data_dict['IS 593']=[]
data_dict['IS 601']=[]
data_dict['IS 612']=[]
data_dict['IS 613']=[]
data_dict['IS 614']=[]
data_dict['IS 616']=[]
data_dict['IS 650']=[]
data_dict['IS 661']=[]
data_dict['IS 664']=[]
data_dict['IS 676']=['IS 663','CS 673']
data_dict['IS 677']=[]
data_dict['IS 682']=[]
data_dict['IS 683']=[]
data_dict['IS 686']=[]
data_dict['IS 698']=[]
data_dict['IS 700']=[]
data_dict['IS 700B']=[]
data_dict['IS 700C']=[]
data_dict['IS 701']=[]
data_dict['IS 701B']=[]
data_dict['IS 701C']=[]
data_dict['IS 725']=[]
data_dict['IS 764']=[]
data_dict['IS 766']=[]
data_dict['IS 776']=[]
data_dict['IS 785']=[]
data_dict['IS 786']=[]
data_dict['IS 790']=[]
data_dict['IS 790D']=[]
data_dict['IS 790E']=[]
data_dict['IS 790F']=[]
data_dict['IS 791']=[]
data_dict['IT 610']=[]
data_dict['IT 635']=[]

**Data Dictionary** 

In [ ]:
data_dict

In [ ]:
#removing the nan values
del data_dict['nan']

In [ ]:
#loading  University-1 graduate courses csv file
Univ1_graduate_dataset=pd.read_csv('Univ1_graduate_courses.csv')

In [ ]:
Univ1_graduate_dataset.head()

In [ ]:
#removing unicode characters from course
Univ1_graduate_dataset['course']=Univ1_graduate_dataset['course'].str.replace('¬†',' ')

In [ ]:
#removing the Unnamed: 3 column from the dataset
Univ1_graduate_dataset.drop('Unnamed: 3',axis=1,inplace=True)

In [ ]:
#cleansing of data
def clean_text(text):
    #Convert to lowercase to maintain consistency
    text = text.lower()
    return text

In [ ]:
#Genrating the frequent words
def gen_freq(text):
    #to store the list of words
    word_list = []

    for tw_words in text:
        word_list.extend(tw_words)
    #Creating word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    return word_freq

In [ ]:
#core courses list for ms in cyberscecurity
cyber_core_list=['CS 608','CS 645','CS 646','CS 647','CS 656','CS 696']

In [ ]:
#elective courses list for ms in cybersecurity
cyber_elective_list=['CS 633','CS 634','CS 643','CS 648','CS 652','CS 660','CS 673','CS 678','CS 680','CS 684','CS 708','CS 755','IS 680','IS 681','IS 682','IS 687','IT 620','IT 640','ECE 636','MGMT 688','MGMT 691','CS 610','CS 630','CS 631']

In [ ]:
#complete courses list for cybersecurity
cyber_courses_list=cyber_core_list + cyber_elective_list

In [ ]:
#to compile one dictinary  for core and elective course where 1 will denote course is core and 0 will denote elective
cyber_dict={}
for i in range(len(cyber_courses_list)):
  key=cyber_courses_list[i]
  if key in cyber_core_list:
    cyber_dict[key]=[1,0]
  else:
    cyber_dict[key]=[0,1]

In [ ]:
cyber_dict

In [ ]:
#creating dictionary for cybersecurity courses and prerequsite
def prereq_dict(data_dict,course_list):
  course_dict={}
  for i in range(len(course_list)):
    key=course_list[i]
    if key in data_dict:
      course_dict[key]=data_dict.get(key)
    else:
      course_dict[key]=[]
  return course_dict

In [ ]:
#cybersecurity  prereq dict with course and prerequisite information
cyber_prereq_dict=prereq_dict(data_dict,cyber_courses_list)

In [ ]:
cyber_prereq_dict

In [ ]:
#Adding double brackets for courses with prereqs in 'and' condition and correcting other courses prereqs
cyber_prereq_dict['ECE 636']=['CS 656']
cyber_prereq_dict['CS 643']=['CS 656','CS 644']
cyber_prereq_dict['CS 646']=['CS 656']
cyber_prereq_dict['CS 647']=['CS 645','CS 646','CS 696']
cyber_prereq_dict['CS 708']=[['CS 608', 'CS 645', 'CS 696']]
cyber_prereq_dict['IT 620']=[]
cyber_prereq_dict['CS 648']=['CS 656']

In [ ]:
cyber_prereq_dict

In [ ]:
#load Univ1 graduate dataset to get the course description
#compare the dataset with cyber courses list and extract them
cyber_df=Univ1_graduate_dataset.loc[Univ1_graduate_dataset['course'].isin(cyber_courses_list)]

In [ ]:
cyber_df.head()

In [ ]:
#Add 3 missing courses  from the cybersecurity courses list in the cyber dataframe
listOfSeries = [pd.Series(['ECE 636','Computer Networking Laboratory','Prerequisites: ECE 637 or CS 656. This course provides students with hands on training regarding the design, troubleshooting, modeling and evaluation of computer networks. In this course, students are going to experiment in a real test-bed networking environment, and learn about network design and troubleshooting topics and tools such as: network addressing, Address Resolution Protocol (ARP), basic troubleshooting tools (e.g. ping, ICMP), IP routing (e,g, RIP), route discovery (e.g. traceroute), TCP and UDP, IP fragmentation and many others. Student will also be introduced to the network modeling and simulation, and they will have the opportunity to build some simple networking models using the OPNET modeling tool and perform simulations that will help them evaluate their design approaches and expected network performance.'], index=cyber_df.columns ) ,
                pd.Series(['MGMT 688','Information Technology Business and the law','Includes historical and constitutional foundations, crimes, and torts in cyberspace, virtual property (patents online, copyrights in digital information, trade secrets in cyberspace, and cybermarks), electronic commerce contracting, electronic commerce, electronic money and the law, and information technology and online infringement of rights of intellectual property.'], index=cyber_df.columns ) ,
                pd.Series(['MGMT 691','Legal and Ethical issues','Explores the legal and ethical responsibilities of managers. Analyzes extent to which shareholders should be allowed to exercise their legitimate economic, legal, and ethical claims on corporate managers; extent of regulation of a particular industry, individual rights of the employee and various corporate interests, and corporate responsibility to consumers, society, and conservation of natural resources and the environment.'], index=cyber_df.columns ) ]

In [ ]:
# Pass a list of series to the append() to add multiple rows
cyber_df = cyber_df.append(listOfSeries , ignore_index=True)

In [ ]:
#to reset the index 
cyber_df=cyber_df.reset_index(drop=True)

In [ ]:
cyber_df

In [ ]:
#converting the course decsription and course_name column data into string format
cyber_df['course description']=cyber_df['course description'].apply(str)
cyber_df['course_name']=cyber_df['course_name'].apply(str)

In [ ]:
#applying the cleaning function to the course_name column
cyber_df['course_name']=cyber_df['course_name'].apply(clean_text)

In [ ]:
#to get the nouns from course name
def find_noun(keyword):
    tokens = nltk.word_tokenize(keyword)
    tagged = nltk.pos_tag(tokens)
    noun = [w for w,t in tagged if "NN" in t]
    return noun

cyber_df['Nouns']=cyber_df['course_name'].apply(find_noun)

#To get  tokenized words from the text
def getting_nouns(text):
  text_blob_object=TextBlob(text)

  return text_blob_object.words


cyber_df['words']=cyber_df['course_name'].apply(getting_nouns)

In [ ]:
cyber_df

In [ ]:
#removing stopwords using nltk library
cyber_df['words'] = cyber_df['words'].apply(lambda x:[item for item in x if item not in stop])

In [ ]:
#converting the words column into a list
list_of_topics=cyber_df['words'].tolist()
list_of_topics

In [ ]:
#making a single list
merged=list(itertools.chain.from_iterable(list_of_topics))

#removing the duplicates
cyber_topics=list(dict.fromkeys(merged))

In [ ]:
print(*cyber_topics,sep=",")

In [ ]:
#Update function for feature vector update at a node
def update_feature_vector(feature_vector,topic_vector,course_list):
  for i in course_list[0]:
    for j in range(len(topic_vector)):
      if i==topic_vector[j]:
         feature_vector[j]=1
  return feature_vector 

In [ ]:
#creating feature dictionary with course as key and topics as values
from tqdm import tqdm
feature_cyber_dict={}
cyber_courses_list=list(cyber_df['course'])
for i in tqdm(cyber_courses_list):
  temp = list(cyber_df[cyber_df['course']== i ]['words'])
  feature_vector_cyber =[0]*len(cyber_topics)
  feature_vector_cyber = update_feature_vector(feature_vector_cyber,cyber_topics,temp)
  feature_cyber_dict[i]=feature_vector_cyber

In [ ]:
#printing the dictionary
for i,j in feature_cyber_dict.items():
  print('{} : {}'.format(i,j))

In [ ]:
edges=[]
for course in cyber_courses_list:

  CourseId=cyber_courses_list[:]
  CourseId.remove(course)

  for i in CourseId:
    edges.append((i,course))
 
for course in cyber_courses_list:
   edges.append(('initial' , course))

       
G = nx.DiGraph()
G.add_edges_from(edges) 
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G,pos, cmap=plt.get_cmap('jet'),node_size = 1000,node_color="lightblue")
nx.draw_networkx_edges(G,pos,edge_color='b', edge_cmap=plt.cm.Blues,arrows=True,arrowstyle="->",arrowsize=10)
nx.draw_networkx_labels(G,pos)

plt.show()

In [ ]:
nodes={}
i=0
for i,node in enumerate(G.nodes()):
  nodes[i]=node

In [ ]:

def getKeysByValue(dictOfElements, valueToFind):
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            key=item[0]
    return  key

In [ ]:
#Collect all possible actions 
actionVectors={}
for edge in edges:
  actionVectors[edge]=cyber_dict[edge[1]]

In [ ]:
#Function to find key based on value in node dictionary
def findKey(course):
  return list(nodes.keys())[list(nodes.values()).index(course)]

In [ ]:
''''Bitwise addition of two lists '''

def addBinary(l1,l2):
  '''Adds two binary lists'''
  if (len(l1) != len(l2)):
    return "Lists must be same length"
  l3 = []
  for i in range(len(l1)):
    if(l1[i]!=l2[i]):
      l3.append(l1[i]+l2[i])
    elif(l1[i]>l2[i]):
      l3.append(l1[i])
    else:
      l3.append(l2[i])
  return l3

In [ ]:
import numpy
import statistics 

def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]



In [ ]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list
  

In [ ]:
''' Returns the maximum number of consecutive numbers that occured in a list '''

def consecutive(list_of_element,element):
    
    list_of_index=get_index_positions(list_of_element, element)
    
    occurance=[]
    count=1
    i=0
    if (len( list_of_index)>=2):
      while i < (len(list_of_index)-1):
        if (list_of_index[i+1] == list_of_index[i]+1):
          count+=1
          i+=1
        else:
          occurance.append(count)
          i+=1
          count=1
      occurance.append(count)

      maximum=max(occurance)
      length=len(list_of_element)
      sum_list=sum(occurance)
      occur=(sum_list * maximum) / length
    elif (list_of_index==[]):
      occur=0
    else:
      occur=list_of_element[list_of_index[0]]


    return occur

In [ ]:
def prereq(List, course):


  if (cyber_prereq_dict[course]!=[]):
    if (type(cyber_prereq_dict[course][0])==list):
      for j in cyber_prereq_dict[course][0]:
        if ((j in List)==False or (len(List)- get_index_positions(List,j)[0]) <2 ):
          return 0 
      return 1
    else:
      for j in cyber_prereq_dict[course]:
        if ((j in List)==True  and (len(List)- get_index_positions(List,j)[0])>2 ):
          return 1
      return 0

  else:
    return 1

In [ ]:
def prereq1(List, number):


  if (cyber_prereq_dict[nodes[number]]!=[]):
    if (type(cyber_prereq_dict[nodes[number]][0])==list):
      for j in cyber_prereq_dict[nodes[number]][0]:
        if ((findKey(j) in List)==False):
          return 0 
      return 1
    else:
      for j in cyber_prereq_dict[nodes[number]]:
        if ((findKey(j) in List)==True):
          return 1
      return 0

  else:
    return 1

In [ ]:
cyber={}
for key in actionVectors:
  if (actionVectors[key]==[1,0]):
    cyber[key]="core"
  else:
    cyber[key]="elective"

In [ ]:
#Class that defines the Enviornment
#Our enviornment follows a graph based model of a data set

class Env:
  def __init__(self,input_graph,goal_vect,action_vect,startState):
    self.input_graph = input_graph #NetworkX graph
    self.goal_vect = goal_vect #Goal Vector/Features
    self.action_vect = action_vect # Action Vectors Dictionary
    self.startState = startState #Integer representing the starting state 
   

  
  def step(self,state1,action1,state1Vect, stateList,w1,w2,delta,beta): # to take the action 
    done = False
    
    action1Vect = self.action_vect[action1][:]
    
    #Take a specified action and get new state 
    if (action1Vect==[1,0]):
      state1Vect.append("core")
    else:
      state1Vect.append("elective")
    state2Vect = state1Vect[:]

    if (len(state2Vect)==len(self.goal_vect[0])): #Have same num of courses as the goal vector, so terminate
      done = True
    
    #Calculate reward of the given move
    rewardList=[]
    for i in range(len(self.goal_vect)):
      
      distance=levenshteinDistanceDP(state2Vect[len(state2Vect)-1], self.goal_vect[i][len(state2Vect)-1])
      if distance==0.0:
        stateList[i].append(1)
      else:
        stateList[i].append(0)
      rewardList.append(consecutive(stateList[i],1))

    rewardTotal=0
    if cyber[action1]=="elective":
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w2
    else:
      rewardTotal= delta * statistics.mean(rewardList) +  beta * w1

    state2 = action1[1]
    # Return second state, reward,done flag
    return state2,state2Vect,rewardTotal,stateList,done

 

  def reset (self):
    #This will reset to initial state 
    # We invoke this on each episode 
    return self.startState



In [ ]:
#Define function to choose action given a state, as per policy of our paper
#Policy is applied to the seed state on each iteration

def choose_action(state,stateUTSoFar,topicVector,goalVector,cyber,possible_states,stateVector, stateList,w1,w2,course_vector,delta,beta):
  
  ideal=[1]*61
  possible_actions={}
   # Get list of possible actions 
  for action in cyber.keys():
    if (action[0] == state and (action[1] in possible_states)== True):
      
      if (all(i == 0 for i in feature_cyber_dict[action[1]])):
        continue
      else:
        a=addBinary(topicVector,feature_cyber_dict[action[1]])
        b=1 - spatial.distance.cosine(topicVector, ideal)
        c=1 - spatial.distance.cosine(a, ideal)
        if (c-b>0.0025): # This is cosine threshold to choose action
        
          possible_actions[action] = cyber[action][:]
    else:
      continue
  
  stateL=[]
  utility={}
  for action in possible_actions:
    #first statevector value is core
    state_vect=stateVector[:]
    
    
    state_vect.append(possible_actions[action])
    
    if (len(state_vect)<11):
      List=[]
      
      state_list=copy.deepcopy(stateList)
      for i in range(len(goalVector)):
        distance=levenshteinDistanceDP(state_vect[len(state_vect)-1], goalVector[i][len(state_vect)-1])
        
        if distance==0:
          state_list[i].append(1)
        else:
          state_list[i].append(0)
        
        List.append(consecutive(state_list[i],1))
      #masked_List = numpy.ma.masked_equal(List, 0.0, copy=False)
      currStateUT=0

      if cyber[action]=="elective":
        currStateUT= (delta * statistics.mean(List) + beta * w2) * prereq(course_vector,action[1])
      else:
        currStateUT= (delta * statistics.mean(List) + beta * w1) * prereq(course_vector,action[1])
      
      sess_utility  = currStateUT + stateUTSoFar 
      utility[action]=sess_utility
      
      
    else:
      break

  
  candidate=[]
  if (utility!={}):
    maximum=max(utility.values())
    for i in utility:
      if (utility[i]==maximum):
        candidate.append(i)
    chosen_action = random.choice(candidate)
    stateUTSoFar=utility[chosen_action]
 
  else:
    chosen_action=()
  
  if (chosen_action==()):
      topicVect2=topicVector
  else:
      topicVect2=addBinary(topicVector, feature_cyber_dict[chosen_action[1]])  


  return chosen_action , stateUTSoFar , topicVect2

In [ ]:
#SARSA
def update(state, state2, action, action2,reward,gamma,alpha): 
    action=action[1]
    action2=action2[1]
    predict = Q[findKey(state), findKey(action)] 
    target = reward + gamma * Q[findKey(state2), findKey(action2)] 
    Q[findKey(state), findKey(action)]  = Q[findKey(state), findKey(action)]  + alpha * (target - predict)

In [ ]:
#Initialize parameters (Set as per preferences)

total_episodes = 100

alpha = 0.95
gamma = 0.75
  
#Initializing the Q-matrix 
numOfStates = G.number_of_nodes() 
Q = np.matrix(np.zeros(shape =(numOfStates, numOfStates)))


#cyber security ideal compositions

idealVect=[["core","elective","core","core","elective","core","elective","core","core","elective"],
           ["core","core","elective","core","elective","core","elective","core","elective","core"],
           ["elective","core","core","elective","core","core","elective","core","elective","core"]]

#Set up graph enviornment for agent 
startState='initial'
env = Env(G, idealVect,actionVectors,startState)

In [ ]:
def choose_first_action(state,stateUTSoFar,topicVector,goalVector,actionVectors,w1,delta,beta):

  ideal=[1]*61

  ''' the first action can be 'CS 608' , 'CS 645' or 'CS 656'. '''
  chosen_action= ('initial' , 'CS 608') # Start Node (Must be CORE course)
  state_vect=["core"]
  List=[]

  for i in range(len(goalVector)):
    distance=levenshteinDistanceDP(state_vect[0], goalVector[i][0])
    if distance==0:
      List.append(1)
    else:
      List.append(0)
  

  currStateUT = delta * statistics.mean(List) + beta * w1

  topicVect1=addBinary(topicVector, feature_cyber_dict[chosen_action[1]])


  return chosen_action , stateUTSoFar+currStateUT , topicVect1

In [ ]:
#Do learning. Episode here is session
import copy
for episode in range(total_episodes):
    print("####################Episode##################=",episode)
    states=cyber_courses_list[:]
    reward = 0
    #core course weight
    w1=0.4
    #elective course weight
    w2=0.6

    #ideal composition weight
    delta=0.4
    beta=0.6

    stateUTSoFar = 0 
    state1 = env.reset()
    topicVect=[0]*61
    course_vector=[]
    action1,stateUTSoFar,topicVect1= choose_first_action(state1,stateUTSoFar,topicVect,idealVect, actionVectors,w1,delta,beta)
    state1Vect = [] # Make it length of num of ideal features 
    course_vector.append(action1[1])
    stateList1=[[]]*len(idealVect)
    while True: 
     
        #Getting the next state 
       
        state2, state2Vect ,reward, stateList2, done = env.step(state1,action1,state1Vect,stateList1,w1,w2,delta,beta)
        states.remove(state2)

       
        #Choosing the next action 
        stateList3=copy.deepcopy(stateList2)
        action2,stateUTSoFar, topicVect2= choose_action(state2,stateUTSoFar,topicVect1,idealVect,cyber, states, state2Vect, stateList3,w1,w2,course_vector,delta,beta) 
        
        if (action2==()):
          break
        else:
          course_vector.append(action2[1])
          
        #Updating the Q table
        update(state1, state2, action1, action2, reward,gamma,alpha) 
        
        state1 = state2 
        action1 = action2 
        state1Vect = state2Vect
        topicVect1=topicVect2
        stateList1=stateList2
        
        #If at the end of learning process, start another session
        if (done==True or states==[]): 
            break

In [ ]:
#Print Q Table as Dataframe
Qdf = pd.DataFrame(Q)
Qdf

In [ ]:
#Traverse the Q table
def getBestPath(Qdf,startState):
  i = getKeysByValue(nodes,startState)
  loc = Qdf.iloc[i].idxmax()

  path =[]
  path.append(loc)
  while len(path) != 10:

    posOfMax = Qdf.iloc[loc].idxmax()
    if ((posOfMax in path )==True or prereq1(path,posOfMax)==0):
      posOfMax_2=Qdf.iloc[loc,~Qdf.columns.isin(path)].idxmax()
      path.append(posOfMax_2) 
      loc=posOfMax_2  
    else:
      path.append(posOfMax)
      loc=posOfMax 
  return path

In [ ]:

print("Best sequence of courses to get a master degree in Cyber Security is: \n")
Seq=getBestPath(Qdf,startState)

Sequence={}
for i in Seq:
  Sequence[nodes[i]]=cyber_dict[nodes[i]]


for key in Sequence:
  if Sequence[key]==[0,1]:
    Sequence[key]='elective'
  else:
    Sequence[key]='core'

for i,j in Sequence.items():
  print('{} : {}'.format(i,j))

print('Number of recommended courses=',len(Sequence))

In [ ]:
#Get the core elective list 
lst = [j for i,j in Sequence.items()]
lst

In [ ]:
#Calculating the score

def compare(List):
  result=[]
  for i in idealVect:
    similarity=[]
    for j in range(len(List)):
      if (levenshteinDistanceDP(List[j],i[j])==0.0):
        similarity.append(1)
      else:
        similarity.append(0)
    result.append(consecutive(similarity,1))
  return result

max(compare(lst))

In [ ]:
#Check the prerequisites condition
''' if all 1's means all prerequisites condition are satisfied '''
a=[]
for i in Sequence:
  a.append(i)
for i in a:
  print(i, prereq(a,i))